In [ ]:
import numpy as np 
import pandas as pd 
 

import os
from PIL import Image as PILImage

import pickle

import glob
import tensorflow.keras.applications.resnet50 as resnet

import IPython.display 
#import keras 
from keras.preprocessing.image import array_to_img 
from keras.preprocessing import image 
 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras import backend as K

K.set_image_data_format('channels_last')
print(keras.__version__, tf.__version__)

from sklearn import *

In [ ]:
IMG_WIDTH = 300
IMG_HEIGHT = 300 
NR_CHANNELS = 3
TOTAL_INPUTS = NR_CHANNELS * IMG_HEIGHT * IMG_WIDTH

In [ ]:
imglist_test =glob.glob(os.sep.join(["..","/input/","plant-pathology-2021-fgvc8/","test_images/", '*.jpg']))
len(imglist_test)

In [ ]:
# For Xim and Xraw for Resnet50 Model input


Xim_test = np.zeros((len(imglist_test), 300, 300, 3), dtype=np.float32)

for i,img_path in enumerate(imglist_test):
    img = image.load_img(img_path, target_size=(300, 300))

    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = resnet.preprocess_input(x)
    Xim_test[i,:] = x
 
print(Xim_test.shape)

In [ ]:
print(Xim_test[0])

In [ ]:
# the customized model

model_f = load_model('../input/resnet50-customized1-model/abc.h5')
# compute the features
X_test = model_f.predict(Xim_test)
print(X_test.shape)

In [ ]:
print(X_test)

In [ ]:
training_csv = pd.read_csv('../input/plant-pathology-2021-fgvc8/train.csv')
training_class = np.array([])
for labels in pd.unique(training_csv['labels']):
    training_class = np.append(training_class,labels.split())
tagnames = np.unique(training_class)

In [ ]:
# convert binary class vector into a list of tags
def class2tags(classes, tagnames):
    tags = []
    for n in range(classes.shape[0]):
        tmp = []
        for i in range(classes.shape[1]):
            if classes[n,i]:
                tmp.append(tagnames[i])
        tags.append(" ".join(tmp))
    return tags

In [ ]:
test_predclass=[]
# convert score into binary class 0 or 1.  
test_predclass = X_test > 0.2

# convert to tags
test_predtags = class2tags(test_predclass, tagnames)

In [ ]:
import gc


del test_predclass
gc.collect()

In [ ]:
test_img_path = '../input/plant-pathology-2021-fgvc8/test_images'
def write_path():
    import csv
    tmp=[]
    for path in os.listdir(test_img_path):
        row = [path]
        tmp=np.hstack((tmp, row))    
    return tmp

In [ ]:
df1=pd.DataFrame(write_path(), columns =['image'])
df1

In [ ]:
df2 = pd.DataFrame(test_predtags, columns =['labels'])
df2

In [ ]:
pd.df=pd.concat([df1, df2], axis=1)
pd.df

In [ ]:
pd.df.to_csv(r'./submission.csv', index = False)